### Load Packages

In [1]:
import pandas as pd
import numpy as np
import os

pd.options.mode.chained_assignment = None

### Import Excel Files

In [2]:
folder = r'C:\Users\mazhang\Documents\Jupyter Notebook\2021 Monthly Closed Invoice'
files = os.listdir(folder)

excel_list = []
for file in files:
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder, file), sheet_name = None)
        excel_list.append(df)

C:\Users\mazhang\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


### Generate Half-Monthly Report with Accounting ID, Facility and  Bill Pay Category, etc.

In [3]:
df_list = []

for excel in range(len(excel_list)):
    
    df_header = excel_list[excel]['Header']
    df_line = excel_list[excel]['Line']
    df_detail = excel_list[excel]['Detail']

    df_select_header = df_header[['Accounting_CustomerID', 'CustomerID', 'DocNo', 'Facility', 'Bill To']]
    df_select_header['Billing Period Start'] = df_detail['Header Billing Period Start']
    df_drop_dup_header = df_select_header.drop_duplicates(subset = ['Accounting_CustomerID', 'DocNo', 
                                                                    'Facility', 'Bill To'])
    
    df_select_line = df_line[['CustomerID', 'Facility', 'Bill Pay Category', 'Bill To']]
    df_drop_dup_line = df_select_line.drop_duplicates(subset = ['Facility', 'Bill To', 'Bill Pay Category'], 
                                                      keep = 'last')

    df_combine_header_line = pd.merge(df_drop_dup_line, df_drop_dup_header, how = 'left', 
                                      on = ['Facility', 'Bill To']).T.drop_duplicates().T
    df_combine_rename = df_combine_header_line.rename(columns = {"CustomerID_x": "CustomerID"})
    df_combine_half_month = df_combine_rename.reindex(columns = ['Accounting_CustomerID', 'CustomerID', 'DocNo', 
                                                                 'Facility', 'Bill Pay Category', 
                                                                 'Bill To', 'Billing Period Start'])
    print(df_combine_half_month.shape)
    
    df_list.append(df_combine_half_month)

(337, 7)
(315, 7)
(343, 7)
(379, 7)
(395, 7)
(418, 7)
(408, 7)
(379, 7)
(411, 7)


### Generate Quarter Report with Accounting ID, Facility and  Bill Pay Category, etc.

In [4]:
df_combine_list = []
df_combine_quarter = df_list[0]

for df in range(len(df_list)):
    
    df_combine_quarter = pd.merge(df_combine_quarter, df_list[df], how = 'outer', 
                                  on = ['Accounting_CustomerID', 'CustomerID', 'DocNo', 
                                        'Facility', 'Bill Pay Category', 'Bill To'])
    print(df_combine_quarter.shape)
    
    df_combine_list.append(df_combine_quarter)
    
df_output_init = df_combine_list[-1]
df_output = df_output_init.T.drop_duplicates().T
df_output.rename({'Billing Period Start_x':'Billing Period Start', 
                'Billing Period Start_y':'Billing Period Start'}, axis = 'columns', inplace = True)
print(df_output.shape)

(337, 8)
(652, 9)
(995, 10)
(1374, 11)
(1769, 12)
(2187, 13)
(2595, 14)
(2974, 15)
(3385, 16)
(3385, 15)


### Export Quarter Report As Excel File

In [5]:
# df_output.to_excel("2021 Monthly Report.xlsx", index = False, header = True)